**OBLIGATORIO FINAL G448 - Data Wrangling**
<br>**INTEGRANTES:**
<br>
+ Theo Pereira, 5703834-3, theopereira2001@gmail.com
+ Carlos Píriz, 4298784-4, piriz.carlos@gmail.com
+ Gabriel	Better,3138871-2, gabrielbetter@gmail.com

In [472]:
### Importacion de librerias ###

# Sistema
import os

# Tratamiento de datos
import pandas as pd
import numpy as np
import datetime as dt
import math

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

# Visualizacion 
import seaborn as sns
import matplotlib as mpl
from matplotlib import ticker
from matplotlib.ticker import FuncFormatter
from babel.numbers import format_currency
import matplotlib.pyplot as plt
%matplotlib inline

**CONSIGNAS**
<br>
El objetivo general es realizar un análisis exploratorio de los anuncios inmobiliarios de
algunas provincias de la Patagonia Argentina reportados por el portal Airbnb.
<br>
Los datos están conformados por cuatro archivos csv:
+ Neuquen.csv
+ Rio_Negro.csv
+ Chubut.csv
+ Tierra_del_Fuego.csv

**<h1>1. Carga de datos y armado del dataset</h1>**

In [473]:
# Carga de archivos de datos de chubut
df_chubut = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Chubut.csv')
df_chubut["provincia"]="CHUBUT"
df_chubut.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
87,903126420705158954,Downtown and Beach with seaview,entire_home,Nuevo,Puerto Madryn,-42.767130,-65.032730,"['2 camas', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$34 USD,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
63,29591102,Habitación en pleno centro para 1 persona,private_room,"5,0 (7)",Trelew,-43.250099,-65.311165,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
18,53472591,On the Spot!(5pax) Pto Madryn - centro 50m del...,entire_home,"4,77 (44)",Puerto Madryn,-42.767826,-65.033134,"['3 camas', '2 dormitorios', '1 baño', '2 cama...",2023-07-01,2023-07-06,NaN,$61 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
107,39340707,"""Cerros Nevados"" departamento tipo loft.",entire_home,"4,96 (27)",Trevelin,-43.079100,-71.461710,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
41,787139916703144070,"Luz, confort y garaje, 2 del mar",entire_home,"5,0 (10)",Puerto Madryn,-42.767240,-65.037240,"['1 cama', 'Monoambiente', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
33,37128706,Nuestro hogar Pirámides,entire_home,"4,9 (20)",Puerto Pirámides,-42.567490,-64.270210,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-10,2023-07-16,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
245,13941382,Cabañas MARI MARI (PUELO),entire_home,NaN,El Hoyo,-42.057100,-71.541730,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
206,914649068770847086,Alquiler temporario,entire_home,Nuevo,Comodoro Rivadavia,-45.842565,-67.473948,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$25 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
118,887853891920643116,"Luma, tu lugar feliz en el sur",entire_home,Nuevo,Esquel,-42.917491,-71.294685,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,$34 USD,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
93,709585936691929156,"Phira rojo, habitación con vista al mar",private_room,"4,73 (11)",Puerto Pirámides,-42.570180,-64.278739,"['1 cama', '1 dormitorio', '1 baño privado']",2023-08-28,2023-09-02,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [474]:
# Carga de archivos de datos de Neuquen
df_neuquen = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Neuquen.csv')

df_neuquen["provincia"] = "NEUQUEN"

df_neuquen.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
146,45759058,Departamento amoblado a nuevo,entire_home,"4,73 (11)",Cipolletti,-38.942960,-67.982410,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$57 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
199,896280194416783327,Aires de la Costa,entire_home,Nuevo,Neuquén,-38.974180,-68.043335,"['2 camas', 'Monoambiente', '1 baño', 'Estacio...",2023-07-01,2023-07-06,NaN,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
229,16691196,30% de descuento. Casa de dormitorios completa...,entire_home,"4,41 (22)",Neuquén,-38.932810,-68.152090,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$99 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
255,791822527552782007,Compras en Dpto Alto Comahue,entire_home,NaN,Neuquén,-38.943280,-68.064730,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-01,2023-07-06,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
125,863566985727115033,Dúplex Hermoso y Luminoso!,entire_home,NaN,Neuquén,-38.921061,-68.080196,"['No hay evaluaciones', '3 camas', '3 dormitor...",2023-07-01,2023-07-07,$94 USD,$118 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
184,865981864850437447,"Mono ambiente ""El Vasco""",private_room,Nuevo,Neuquén,-38.947928,-68.087324,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$27 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
42,28673402,Limay Bed&Breakfast Séxtuple,entire_home,"5,0 (6)",Neuquén,-38.979370,-68.061220,"['6 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-01,2023-07-08,NaN,$156 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
122,26215495,Cómodo departamento en el Centro de Cipolletti.,entire_home,"4,89 (19)",Cipolletti,-38.942310,-67.989640,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-08,$47 USD,$52 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
158,53934589,Casa moderna y tranquila en un amplio terreno,entire_home,Nuevo,Cipolletti,-38.904470,-67.994720,"['No hay evaluaciones', '4 camas', '3 dormitor...",2023-07-01,2023-07-06,NaN,$138 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
89,579026916207068038,Monoambiente nivel ejecutivo zona centro Neuquén.,entire_home,NaN,Neuquén,-38.953239,-68.063423,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$36 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [475]:
# Carga de archivos de datos de Rio Negro
df_rio_negro = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Rio_Negro.csv')

df_rio_negro["provincia"] = "RIO NEGRO"

df_rio_negro.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
79,52109877,Container Studio,entire_home,"5,0 (53)",San Carlos de Bariloche,-41.146350,-71.412320,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-27,2023-09-01,$45 USD,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
49,49107072,Contenedor-loft,entire_home,"4,91 (55)",San Carlos de Bariloche,-41.134950,-71.325710,"['1 cama', '1 dormitorio', '1 baño', '1 cama k...",2023-10-26,2023-11-01,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
40,881218792320627389,Tiny house en el bosque,entire_home,Nuevo,San Martin de los Andes,-40.095194,-71.303187,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
87,42373463,Depto. Vista Nahuel Huapi,entire_home,"4,34 (94)",San Carlos de Bariloche,-41.134370,-71.316880,"['1 cama', 'Monoambiente', '1 baño', '1 sofá c...",2023-08-31,2023-09-05,NaN,$62 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
151,554319517238694179,Cabaña Raulí para 8 con jacuzzi,entire_home,"4,94 (17)",San Carlos de Bariloche,-41.145020,-71.412740,"['7 camas', '4 dormitorios', '1 ,5 baños', '2 ...",2023-09-05,2023-09-11,$157 USD,$185 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
200,6993416,Cabaña Lago Bahía Protegida Costa Lago,entire_home,"4,92 (100)",San Carlos de Bariloche,-41.098340,-71.446220,"['6 camas', '2 dormitorios', '2 baños', 'Estac...",2023-08-12,2023-08-17,NaN,$300 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
113,48254202,Gartehütte,entire_home,"4,96 (50)",San Carlos de Bariloche,-41.110810,-71.425520,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",2023-09-06,2023-09-12,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
261,22978524,Studio Buda. Bonito luminoso y acogedor. H028910,entire_home,"4,92 (61)",San Carlos de Bariloche,-41.138260,-71.325540,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
4,35064808,"Cabaña en el bosque, Bariloche, Patagonia",entire_home,"5,0 (58)",Bariloche,-41.049420,-71.501500,"['3 camas', '2 dormitorios', '1 baño', 'Estaci...",2023-09-04,2023-09-09,NaN,$122 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
32,53404054,Peta Lodge Patagonia II,entire_home,"4,88 (67)",San Carlos de Bariloche,-41.119630,-71.380870,"['3 camas', '1 dormitorio', '2 baños', 'Wifi',...",2023-09-20,2023-09-25,$122 USD,$139 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [476]:
# Carga de archivos de datos de Tierra delFuego
df_tierra_del_fuego = pd.read_csv('https://raw.githubusercontent.com/carlos1680/CPEDWObligatorio/main/Tierra_del_Fuego.csv')

df_tierra_del_fuego["provincia"] = "TIERRA DEL FUEGO"

df_tierra_del_fuego.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
155,20441841,"Acogedora habitación para 4 personas, entrada ...",private_room,"4,82 (73)",Ushuaia,-54.796460,-68.313900,"['4 camas', '2 dormitorios', '1 baño privado',...",2023-07-13,2023-07-20,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
151,53026452,Apartamento con balcón.,entire_home,NaN,Ushuaia,-54.799970,-68.283020,"['No hay evaluaciones', '2 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
192,837061192294336671,Loft del Fin del Mundo,entire_home,"5,0 (3)",Ushuaia,-54.831543,-68.353667,"['3 camas', '1 dormitorio', '1 ,5 baños', 'Wif...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
181,40803781,Shalom del sur 2,entire_home,"5,0 (16)",Ushuaia,-54.822550,-68.331190,"['2 camas', '1 dormitorio', '1 baño', '2 camas...",2023-07-01,2023-07-06,NaN,$36 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
259,738224105784766502,Ruta 3,entire_home,"5,0 (5)",Ushuaia,-54.819590,-68.350290,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-08,2023-07-13,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
66,853572791021667394,Ushuaia54 II,entire_home,"5,0 (3)",Ushuaia,-54.826597,-68.362357,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,$29 USD,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
207,831611894265914063,Ushuaia home,private_room,"4,5 (4)",Ushuaia,-54.821110,-68.326990,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-01,2023-07-06,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
205,903056815345574639,Terrazas del Sur,entire_home,Nuevo,Ushuaia,-54.798545,-68.308509,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-30,2023-08-04,$36 USD,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
189,4654145,DEPARTAMENTO VIDA - LOS CANELOS,entire_home,"4,76 (122)",Ushuaia,-54.807800,-68.324020,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
7,42025621,Lengas Loft,entire_home,NaN,Ushuaia,-54.832146,-68.356553,"['3 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-20,2023-07-27,$22 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [477]:
# Verificacion de columnas para concatenar
(df_chubut.columns == df_neuquen.columns) & (df_rio_negro.columns == df_tierra_del_fuego.columns) & (df_chubut.columns == df_tierra_del_fuego.columns)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True])

In [478]:
# Concatenacion de los dataframes separados por provincias en un unico archivo
df_alquileres = pd.concat([df_chubut, df_neuquen, df_rio_negro, df_tierra_del_fuego])
df_alquileres.sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
129,42084544,Departamento amoblado de un dormitorio 43m2.,entire_home,"4,67 (3)",Río Grande,-53.772110,-67.717980,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$42 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
219,29800413,Ruta de la Patagonia cabaña ecologica 40 #1,entire_home,"4,96 (102)",San Martin de los Andes,-40.312720,-71.380690,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-15,2023-08-20,NaN,$44 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
134,52994498,Petit House II.,private_room,"4,82 (45)",BAX,-54.808320,-68.315950,"['2 camas', '1 dormitorio', 'Wifi', 'Cocina', ...",2023-07-01,2023-07-06,NaN,$37 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
110,813190673021540204,Habitación en casa del bosque.,private_room,"5,0 (6)",San Carlos de Bariloche,-41.133472,-71.310278,"['1 cama', '1 dormitorio', '1 baño', 'Estacion...",2023-08-07,2023-08-12,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
5,42407010,Luminoso Centro + Cochera (50mts RN22),entire_home,"4,56 (94)",BVH,-38.959240,-68.051260,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-07,NaN,$23 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
166,4585169,Casa de montaña serena. A poca distancia en au...,entire_home,"4,93 (14)",El Hoyo,-42.107870,-71.520300,"['3 camas', '3 dormitorios', '1 baño', '1 cama...",2023-09-09,2023-09-15,NaN,$80 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
87,27328190,Ushuaia Departamento,entire_home,"4,98 (94)",Ushuaia,-54.819510,-68.345760,"['4 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-29,2023-08-03,NaN,$36 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
219,49179716,Moderno departamento para 4 personas,entire_home,"4,67 (3)",Cipolletti,-38.948870,-67.983440,"['4 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-08-17,2023-08-22,NaN,$47 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
77,765212466826049144,Mirabeagle 8,entire_home,"4,8 (5)",Ushuaia,-54.837470,-68.364630,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
98,30622116,"Cabaña en el bosque, cerca del centro de Ski",entire_home,"4,84 (69)",San Carlos de Bariloche,-41.149770,-71.407050,"['1 baño', '1 cama queen', 'Estacionamiento gr...",2023-07-07,2023-07-12,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [479]:
# Reseteo del index, permitiendo duplicados porque aun no sabemos si son unicos los valores
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= True)
# df_alquileres.reset_index(names = 'id_alojamiento', drop=True,inplace=True,allow_duplicates= False)
df_alquileres.reset_index(drop=True,inplace=True)

*Verificacion de filas de los dataframes*

In [480]:
# Verificacion de la cantidad de filas de cada dataframe
print("La cantidad de filas del dataframe de Chubut es:"+str(df_chubut.shape[0]))
print("La cantidad de filas del dataframe de Neuquen es:"+str(df_neuquen.shape[0]))
print("La cantidad de filas del dataframe de Rio Negro es:"+str(df_rio_negro.shape[0]))
print("La cantidad de filas del dataframe de Tierra del fuego es:" + str(df_tierra_del_fuego.shape[0]))
print("La cantidad de filas del dataframe unificado de alquileres es:" + str(df_alquileres.shape[0]))


La cantidad de filas del dataframe de Chubut es:270
La cantidad de filas del dataframe de Neuquen es:270
La cantidad de filas del dataframe de Rio Negro es:270
La cantidad de filas del dataframe de Tierra del fuego es:270
La cantidad de filas del dataframe unificado de alquileres es:1080


*Verificacion de las columnas de los dataframes*

In [481]:
# Verificacion de la cantidad de columnas de cada dataframe
print("La cantidad de columnas del dataframe de Chubut es:" +
      str(df_chubut.shape[1]))
print("La cantidad de columnas del dataframe de Neuquen es:" +
      str(df_neuquen.shape[1]))
print("La cantidad de columnas del dataframe de Rio Negro es:" +
      str(df_rio_negro.shape[1]))
print("La cantidad de columnas del dataframe de Tierra del fuego es:" +
      str(df_tierra_del_fuego.shape[1]))
print("La cantidad de columnas del dataframe unificado de alquileres es:" +
      str(df_alquileres.shape[1]))

La cantidad de columnas del dataframe de Chubut es:18
La cantidad de columnas del dataframe de Neuquen es:18
La cantidad de columnas del dataframe de Rio Negro es:18
La cantidad de columnas del dataframe de Tierra del fuego es:18
La cantidad de columnas del dataframe unificado de alquileres es:18


*Verificacion general e inicial del dataframe unificado*

In [482]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1080 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1080 non-null   int64  
 1   name              1080 non-null   object 
 2   category          1080 non-null   object 
 3   rating            886 non-null    object 
 4   city              1076 non-null   object 
 5   latitude          1080 non-null   float64
 6   longitude         1080 non-null   float64
 7   characteristics   1080 non-null   object 
 8   check_in          1080 non-null   object 
 9   check_out         1080 non-null   object 
 10  price_discounted  201 non-null    object 
 11  price_original    1080 non-null   object 
 12  price_qualifier   1080 non-null   object 
 13  cant_resultados   1080 non-null   int64  
 14  fecha_scrapping   1080 non-null   object 
 15  fecha_cambio_bna  1080 non-null   object 
 16  cambio_bna        1080 non-null   float64


In [483]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.describe()

,id_alojamiento,latitude,longitude,cant_resultados,cambio_bna
count,1.080000e+03,1080.000000,1080.000000,1080.000000,1080.000000
mean,3.685341e+17,-44.426201,-68.723924,824.500000,239.919355
std,3.903051e+17,6.164758,2.159715,304.115744,0.000000
min,4.468310e+05,-54.844914,-71.688000,298.000000,239.919355
25%,3.529220e+07,-47.854503,-71.310280,824.500000,239.919355
50%,5.408344e+07,-42.003780,-68.305610,1000.000000,239.919355
75%,7.872495e+17,-39.609811,-68.049328,1000.000000,239.919355
max,9.198428e+17,-38.874370,-62.920960,1000.000000,239.919355


In [484]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.duplicated(keep="first").sum()

51

In [485]:
# Informacion de las columnas, sus tipos y faltantes
df_alquileres.isna().sum()

id_alojamiento        0
name                  0
category              0
rating              194
city                  4
latitude              0
longitude             0
characteristics       0
check_in              0
check_out             0
price_discounted    879
price_original        0
price_qualifier       0
cant_resultados       0
fecha_scrapping       0
fecha_cambio_bna      0
cambio_bna            0
provincia             0
dtype: int64

*Conclusiones iniciales*
<br>
+ El dataframe unificado, coincide en cantidad de filas y columnas correspondientes a los dataframes individuales
+ Debemos de corregir algunos tipos de datos de columnas
+ Tenemos algunas columnas para reescalar
+ Tenemos filas duplicadas para revisar
+ Tenemos valores nulos para revisar, en principio para las columnas rating y price_discounted

**<h1>2. Análisis de duplicados</h1>**

*Analisis y tratamiento de duplicados global*

In [486]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres.duplicated(keep="first").sum()

51

In [487]:
#  Listo los valores duplicados
df_alquileres[df_alquileres.duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
54,40755147,La Amarilla Casa de Huéspedes (con cocina) Rut...,entire_home,"4,93 (27)",Epuyén,-42.227509,-71.362289,"['2 camas', '1 dormitorio', '1 ,5 baños', '1 c...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
72,586216477715336949,Encantador dpto. con vista al mar en Puerto Ma...,entire_home,"5,0 (11)",Puerto Madryn,-42.771633,-65.029068,"['3 camas', '2 dormitorios', '2,5 baños', 'Cal...",2023-07-06,2023-07-11,NaN,$131 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
90,37836757,Rincón del Sol,entire_home,"4,4 (5)",Puerto Madryn,-42.751650,-65.053520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$18 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
113,736253141994173324,Refugio Ciprés con tina de agua caliente,entire_home,"5,0 (4)",Trevelin,-42.968260,-71.601320,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
127,21925703,Hermoso Duplex con Patio y Parrilla. (4 pax).,entire_home,"4,81 (74)",Puerto Madryn,-42.787700,-65.023850,"['3 camas', '2 dormitorios', '1 baño', '1 cama...",2023-07-01,2023-07-06,NaN,$57 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
128,644043911246503372,"Los Molles, casa familiar en entorno natural.",entire_home,NaN,Puerto Madryn,-42.735607,-65.100117,"['3 camas', '2 dormitorios', '1 baño']",2023-07-01,2023-07-07,NaN,$28 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
129,883174783781736766,Espacio 2 - Rada Tilly,entire_home,Nuevo,Rada Tilly,-45.925810,-67.575650,"['1 cama', '1 dormitorio', '1 baño']",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
138,53298317,Mar de Olivillos Planta alta,entire_home,"5,0 (12)",Puerto Piramides,-42.569200,-64.273610,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$75 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
162,31081731,"Domos Puelo, para dos, con jacuzzi",entire_home,"4,98 (40)",Lago Puelo,-42.041570,-71.592370,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-08,2023-07-15,NaN,$67 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
182,50704005,Hostel patagónico con vista 360 de la Cordillera,shared_room,"4,86 (7)",Trevelin,-43.071450,-71.454920,"['20 camas', '4 dormitorios', '3,5 baños compa...",2023-07-01,2023-07-06,NaN,$17 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


In [488]:
# Elimino los duplicados, que coinciden en toda la fila
df_alquileres.drop_duplicates(keep="first", inplace=True)
df_alquileres.shape

(1029, 18)

*Analisis y tratamiento de duplicados por columna*

+ Columna 1: id_alojamiento

In [489]:
# Busco si hay duplicados
df_alquileres['id_alojamiento'].duplicated(keep='first').sum()

1

In [490]:
# Identifico filas con esos valores duplicados
df_alquileres[df_alquileres['id_alojamiento'].duplicated(keep='first')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [491]:
# Muestro las filas que corresponden a los valores duplicados, todas
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
648,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


-> Analisis de los valores de las filas duplicadas.

In [492]:
#Verificamos los valores 
df_alquileres.loc[647]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [493]:
# Verificamos los valores
df_alquileres.loc[648]

id_alojamiento                                               53729814
name                                                     Pequeña casa
category                                                  entire_home
rating                                                       5,0 (36)
city                                          San Carlos de Bariloche
latitude                                                    -41.14768
longitude                                                   -71.31827
characteristics     ['1 cama', '1 dormitorio', '1 baño', '1 cama d...
check_in                                                   2023-08-18
check_out                                                  2023-08-23
price_discounted                                                  NaN
price_original                                                $39 USD
price_qualifier                                                 noche
cant_resultados                                                  1000
fecha_scrapping     

In [494]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[647]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi']"

In [495]:
# Verificamos los valores para la variable que tendria fiferencia
df_alquileres.loc[648]["characteristics"]

"['1 cama', '1 dormitorio', '1 baño', '1 cama doble', 'Estacionamiento gratis', 'Wifi', 'Calificado con 5 de 5 en limpieza']"

*ACCION:*
<BR>
*Vemos que seria el mismo registro con excepcion de la serie characteristics, donde tiene agregada comentario de calificacion*
<br>
*Como consideramos que cada valor de id_alojamiento, deberia ser unico, eliminamos el que consideramos repetido y menos se adecua a los valores esperados por serie*

In [496]:
# Descartamos el registro que no tiene agregado el comentario de la calificacion
df_alquileres.drop([648], inplace=True)
df_alquileres[df_alquileres['id_alojamiento'] == 53729814]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
647,53729814,Pequeña casa,entire_home,"5,0 (36)",San Carlos de Bariloche,-41.14768,-71.31827,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-08-18,2023-08-23,NaN,$39 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


+ Columna 2: name

In [497]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['name'].duplicated(keep="first").sum()

1

In [498]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres[df_alquileres['name'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.1333,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


In [499]:
df_alquileres[df_alquileres['name'].str.contains('Vista al Lago en el Bosque - Hab Compartida')]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
712,797077259992479162,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13455,-71.36807,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-11,2023-07-18,NaN,$21 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
794,800041160997364459,Vista al Lago en el Bosque - Hab Compartida,shared_room,"5,0 (8)",San Carlos de Bariloche,-41.13330,-71.36914,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-05,2023-07-10,NaN,$23 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO


*ACCION:*
<br>
*En este caso vemos que los nombre coinciden pero, pueden corresponder a diferentes lugares, de hecho, estan sobre diferentes calles, segun sus coordenadas.*
<br>
*Por tal motivo los dejamos a ambos registros*

+ Columna 3: category

In [500]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['category'].duplicated(keep="first").sum()

1025

In [501]:
# Listo los duplicados segun category
df_alquileres[df_alquileres['category'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
57,52968978,Casa de Campo con Tina de agua caliente Trevelin,entire_home,"4,77 (13)",Trevelin,-42.991430,-71.559460,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-08-11,2023-08-16,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
933,53040595,Departamento de 3 ambientes con vista al Mar.,entire_home,NaN,Ushuaia,-54.793260,-68.237080,"['4 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-02,2023-07-08,$76 USD,$96 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
434,831209328855554547,Casa dos habitaciones,entire_home,NaN,Neuquén,-38.966110,-68.049350,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-09-10,2023-09-17,NaN,$52 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
478,35213605,CÉNTRICO LOFT Alto Comahue Shopping!!!,entire_home,NaN,Q8300MDJ,-38.941870,-68.065130,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$20 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
910,671793929721970610,CÁLIDO DEPARTAMENTO EN ZONA CENTRO,entire_home,"4,8 (5)",Ushuaia,-54.806390,-68.318740,"['3 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-09-05,2023-09-10,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
529,853518125031245252,Alquiler Bahia Manzano Resort,entire_home,NaN,Neuquén,-38.960750,-68.098340,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-01,2023-07-08,NaN,$90 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
1006,847640105410806361,Mini casa,entire_home,NaN,Río Grande,-53.782990,-67.696630,"['2 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-15,2023-07-20,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
972,556497436406730911,Casa en Ushuaia frente al mar,entire_home,"5,0 (3)",Ushuaia,-54.842760,-68.367680,"['7 camas', '4 dormitorios', '3 baños', '1 cam...",2023-08-11,2023-08-16,$300 USD,$400 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
32,882608055979409957,Alto Madryn Vista 270°,entire_home,Nuevo,Puerto Madryn,-42.768744,-65.035268,"['6 camas', '3 dormitorios', '2,5 baños']",2023-07-01,2023-07-06,NaN,$71 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1049,15312978,Casa Del Beagle,entire_home,"4,85 (137)",Ushuaia,-54.801630,-68.282500,"['6 camas', '3 dormitorios', '1 baño', 'Wifi',...",2023-07-01,2023-07-06,NaN,$120 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que los valores de categoria se pueden repetir para los diferentes alquileres*
<br>
*por lo cual, no realizamos ninguna accion en base a esta columna*

+ Columna 4: rating

In [502]:
# Verifico los duplicado
df_alquileres['rating'].duplicated(keep="first").sum()

564

In [503]:
# Listamos algunas filas, repetidas por rating
df_alquileres[df_alquileres['rating'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
325,788539781586101591,Bim 5,entire_home,NaN,Neuquén,-38.948340,-68.060020,"['2 camas', '1 dormitorio', '1 baño', 'Estacio...",2023-07-10,2023-07-15,NaN,$35 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
443,53105728,LOFT RESIDENCIAL ALTO COMAHUE DE COMPRAS!!,entire_home,NaN,Neuquén,-38.941450,-68.063080,"['No hay evaluaciones', '2 camas', 'Monoambien...",2023-07-01,2023-07-06,NaN,$20 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
150,790783682913096478,Dpto Tecka,entire_home,"5,0 (6)",Comodoro Rivadavia,-45.786747,-67.432560,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-07-01,2023-07-06,NaN,$48 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
298,614959671532733604,Santa Genoveva 1 dormitorio Excelentes vistas,entire_home,"5,0 (11)",Neuquén,-38.949740,-68.044680,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-23,2023-07-28,$30 USD,$41 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
488,902210732743493865,Amplia Casa Nivel Gerencial Nqn,entire_home,Nuevo,Neuquén,-38.925618,-68.050926,"['5 camas', '3 dormitorios', '3 baños', 'Estac...",2023-08-07,2023-08-13,NaN,$220 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
822,53848130,Ocean & Mountain View,entire_home,"4,89 (9)",Ushuaia,-54.807163,-68.318520,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,$88 USD,$98 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
532,780550736495180388,Patagonia,entire_home,NaN,Cipolletti,-38.939712,-67.980420,"['No hay evaluaciones', '1 cama', '1 dormitori...",2023-07-01,2023-07-06,NaN,$38 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,NaN,Puerto Madryn,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
490,47372143,Dúplex amoblados de dos dormitorios,entire_home,NaN,Confluencia,-38.954000,-68.203960,"['4 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-01,2023-07-06,NaN,$100 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
317,811637498175849726,Luxury - Babel NQN,entire_home,NaN,Neuquén,-38.952330,-68.063580,"['No hay evaluaciones', '4 camas', '3 dormitor...",2023-07-01,2023-07-08,NaN,$88 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN


In [504]:
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

*ACCION:*
<br>
*Entendemos que los valores de rating se pueden repetir para los diferentes alquileres*
<br>
*Por lo cual, dejamos los registros retpetidos segun esta columna*

+ Columna 5: city

In [505]:
# Verifico los duplicados, que coinciden en toda la fila
df_alquileres['city'].duplicated(keep="first").sum()

947

In [506]:
# Listamos repetidos, por city
df_alquileres[df_alquileres['city'].duplicated(keep="first")]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,NaN,Puerto Madryn,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
2,800749301724167838,Departamento Bella Vista,entire_home,"5,0 (6)",Puerto Madryn,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,"4,91 (70)",Puerto Madryn,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
4,916738743930751566,Front Beach,entire_home,Nuevo,Puerto Madryn,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
5,27877802,Luxury top 11th floor with the best 360 views,entire_home,"4,96 (95)",Puerto Madryn,-42.768330,-65.032100,"['4 camas', '2 dormitorios', '2,5 baños', '1 c...",2023-07-06,2023-07-11,NaN,$122 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,771725397497773450,Casa Fueguina en Centro,entire_home,"4,53 (17)",Ushuaia,-54.803310,-68.302580,"['3 camas', '2 dormitorios', '1 baño', 'Wifi',...",2023-08-14,2023-08-19,$60 USD,$76 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1076,26215568,Hospedaje UshuaiAndo,entire_home,"4,82 (50)",Ushuaia,-54.800790,-68.304230,"['3 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-23,2023-07-28,$45 USD,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1078,51921207,KREN I,entire_home,"4,85 (26)",Ushuaia,-54.806170,-68.317800,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-10,2023-07-15,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna city, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 6: latitude

In [507]:
# Verificamos los duplicados
df_alquileres['latitude'].duplicated(keep="first").sum()

36

In [508]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['latitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
1077,42486711,Yeska C Dpto. Supercéntrico. Casco Histórico,entire_home,"4,88 (8)",Ushuaia,-54.805430,-68.308710,"['1 cama', '1 dormitorio', '1 baño', '1 cama d...",2023-07-01,2023-07-06,NaN,$63 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
791,4469246,Habitación en Casita de madera y adobe. c/desa...,private_room,"4,68 (114)",San Carlos de Bariloche,-41.073160,-71.488690,"['1 cama', '1 dormitorio', '1 baño compartido'...",2023-07-09,2023-07-14,NaN,$40 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
296,661388148387143050,Apart céntrico 2 dorm c/ balcon cochera opci...,entire_home,"5,0 (6)",Neuquén,-38.951410,-68.061740,"['3 camas', '2 dormitorios', '1 ,5 baños', 'Wi...",2023-07-01,2023-07-08,NaN,$50 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
1071,700712866756917764,D) Cuarto doble privado,private_room,"4,85 (13)",Ushuaia,-54.796494,-68.298038,"['2 camas', '1 dormitorio', '1,5 baños compart...",2023-07-01,2023-07-06,NaN,$26 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1031,711709136406452484,F) Cama en cuarto mixto compartido,shared_room,"4,67 (9)",Ushuaia,-54.796494,-68.298038,"['1 cama', '1 dormitorio', '1,5 baños comparti...",2023-07-01,2023-07-06,NaN,$12 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
768,25675737,Departamento completo en el centro de Bariloche,entire_home,"4,93 (153)",San Carlos de Bariloche,-41.134030,-71.308150,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-04,2023-10-11,NaN,$32 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
1024,906355891628807276,Grand House,entire_home,Nuevo,Ushuaia,-54.806780,-68.315870,"['5 camas', '2 dormitorios', '2 baños', 'Wifi'...",2023-07-03,2023-07-09,$140 USD,$175 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
717,49546396,Departamento para 2. Excelente ubicación.,entire_home,"4,89 (53)",San Carlos de Bariloche,-41.134550,-71.299200,"['1 cama', 'Monoambiente', '1 baño', '1 cama q...",2023-07-07,2023-07-13,NaN,$70 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
709,17846750,Cabaña en Bariloche a minutos del Centro,entire_home,"4,89 (108)",San Carlos de Bariloche,-41.127640,-71.358010,"['4 camas', '2 dormitorios', '1 baño', '1 cama...",2023-08-09,2023-08-14,$75 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
982,690413283545175725,Las Aguilas 3,entire_home,"4,67 (9)",Ushuaia,-54.814922,-68.341647,"['4 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-08,2023-07-14,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [509]:
# Vemos un ejemplo
df_alquileres[df_alquileres['latitude'] == -54.803600]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
934,31825003,Departamento con inmejorable ubicación en Ushuaia,entire_home,"4,9 (108)",Ushuaia,-54.8036,-68.30374,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1025,623178032761843625,"departamento céntrico, cálido y cómodo.",entire_home,"4,88 (16)",AIU,-54.8036,-68.32132,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$33 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna latitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 7: longitude

In [510]:
# Verificamos los duplicados
df_alquileres['longitude'].duplicated(keep="first").sum()

20

In [511]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['longitude'].duplicated(keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
789,27130827,MIRADOR I Hermoso Studio Centro de Bariloche,entire_home,"4,7 (84)",San Carlos de Bariloche,-41.134400,-71.317130,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$90 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.770730,-65.029470,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1071,700712866756917764,D) Cuarto doble privado,private_room,"4,85 (13)",Ushuaia,-54.796494,-68.298038,"['2 camas', '1 dormitorio', '1,5 baños compart...",2023-07-01,2023-07-06,NaN,$26 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
948,889733753812060778,Ushuaia Mágica - Ush AustraLis,entire_home,Nuevo,Ushuaia,-54.811890,-68.326950,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-09,2023-07-14,NaN,$45 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1050,828090107927300389,GRUshuaia,entire_home,NaN,Ushuaia,-54.815088,-68.329166,"['No hay evaluaciones', '1 cama', 'Monoambient...",2023-09-13,2023-09-18,$51 USD,$85 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
982,690413283545175725,Las Aguilas 3,entire_home,"4,67 (9)",Ushuaia,-54.814922,-68.341647,"['4 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-08,2023-07-14,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
941,53903647,Skyline 4,entire_home,"4,88 (8)",Ushuaia,-54.807163,-68.318520,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-04,2023-07-09,$88 USD,$98 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
976,48043650,Latitud Sur Ushuaia,entire_home,"4,95 (22)",Ushuaia,-54.810780,-68.327640,"['2 camas', 'Monoambiente', '1 baño', '1 cama ...",2023-07-01,2023-07-06,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
905,579038522149900540,Departamento con hermosa vista a la Bahía,entire_home,"4,81 (21)",Ushuaia,-54.807163,-68.318520,"['3 camas', '1 dormitorio', '1 baño', 'Wifi', ...",2023-07-03,2023-07-08,NaN,$77 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
1044,51921353,KREN II,entire_home,"4,77 (30)",Ushuaia,-54.807290,-68.316100,"['2 camas', 'Monoambiente', '1 baño', 'Wifi', ...",2023-07-01,2023-07-06,NaN,$50 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


In [512]:
# Vemos un ejemplo
df_alquileres[df_alquileres['longitude'] == -65.029470]

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
70,20364091,Departamento Solmar,entire_home,"4,82 (137)",Puerto Madryn,-42.76963,-65.02947,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-10-15,2023-10-20,NaN,$47 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
257,53295849,Preciosa unidad de alquiler de 3 dormitorios e...,entire_home,"4,84 (25)",Puerto Madryn,-42.77073,-65.02947,"['3 camas', '2 dormitorios', '2 baños', '1 cam...",2023-07-03,2023-07-08,NaN,$114 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT


*ACCION:*
<br>
*Entendemos que la columna longitude, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columan 8: characteristics

In [513]:
# Verificamos los duplicados
df_alquileres['characteristics'].duplicated(keep="first").sum()

243

In [514]:
# Listamos algunos de los duplicados
df_alquileres[df_alquileres['characteristics'].duplicated(
    keep="first")].sample(10)

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia
491,814500878441793574,Jardines Departamento,entire_home,NaN,Neuquén,-38.977332,-68.069833,"['1 cama', 'Monoambiente', '1 baño', 'Estacion...",2023-07-01,2023-07-06,NaN,$57 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
763,53572179,"Dpto céntrico, en complejo residencial con pil...",entire_home,"5,0 (38)",San Martin de los Andes,-40.152730,-71.352130,"['1 cama', '1 dormitorio', '1 baño', 'Wifi', '...",2023-07-01,2023-07-06,NaN,$68 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
713,52993593,Habitación privada en San Martín de Los Andes ...,private_room,"4,96 (27)",San Martin de los Andes,-40.133560,-71.297480,"['1 cama', '1 dormitorio', '2 baños compartido...",2023-07-01,2023-07-06,NaN,$30 USD,noche,1000,2023-06-24,2023-05-01,239.919355,RIO NEGRO
192,822302342590154561,"Mamma, Esquel.",entire_home,"4,67 (3)",Esquel,-42.910710,-71.300210,"['1 cama', '1 dormitorio', '1 baño', 'Califica...",2023-07-01,2023-07-06,NaN,$52 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
1016,710023636824621678,Estudio con vista a las montañas,entire_home,NaN,Ushuaia,-54.795320,-68.238110,"['No hay evaluaciones', '2 camas', '1 dormitor...",2023-07-01,2023-07-06,NaN,$69 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
483,833251451686101343,FLOMA Alquiler Temporario,entire_home,NaN,Neuquén,-38.954630,-68.093190,"['No hay evaluaciones', '3 camas', '2 dormitor...",2023-07-09,2023-07-15,NaN,$62 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
233,894686016674794398,Buena Vista Trevelin II,entire_home,Nuevo,Trevelin,-43.072300,-71.458040,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-13,$34 USD,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT
531,833878451226342434,El lugar de Ana - Am. Comahue,entire_home,NaN,Neuquén,-38.953065,-68.065207,"['No hay evaluaciones', '3 camas', '1 dormitor...",2023-07-02,2023-07-08,NaN,$59 USD,noche,298,2023-06-24,2023-05-01,239.919355,NEUQUEN
956,53699993,Moderno estudio con jardín,entire_home,"4,94 (54)",Ushuaia,-54.809040,-68.319680,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-10-23,2023-10-28,NaN,$55 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO
936,40149949,Moderno estudio 3 Ushuaia Center,entire_home,"4,77 (79)",Ushuaia,-54.803800,-68.301960,"['1 cama', 'Monoambiente', '1 baño', 'Wifi', '...",2023-07-10,2023-07-15,NaN,$59 USD,noche,1000,2023-06-24,2023-05-01,239.919355,TIERRA DEL FUEGO


*ACCION:*
<br>
*Entendemos que la columna characteristics, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 9: check_in

In [515]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_in'].duplicated(keep="first").sum()

920

*ACCION:*
<br>
*Entendemos que la columna check_in, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 10: check_out

In [516]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['check_out'].duplicated(keep="first").sum()

916

*ACCION:*
<br>
*Entendemos que la columna check_out, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 11: price_discounted

In [517]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_discounted'].duplicated(keep="first").sum()

938

*ACCION:*
<br>
*Entendemos que la columna price_discounted, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 12: price_original

In [518]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_original'].duplicated(keep="first").sum()

868

*ACCION:*
<br>
*Entendemos que la columna price_original, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 13: price_qualifier

In [519]:
# Verificamos la cantidad de duplicados por esta columna
df_alquileres['price_qualifier'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna price_qualifier, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 14: cant_resultados

In [520]:
# Verificamos los duplicados
df_alquileres['cant_resultados'].duplicated(keep="first").sum()

1026

*ACCION:*
<br>
*Entendemos que la columna cant_resultados, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

Columna 15: fecha_scrapping

In [521]:
# Verificamos los duplicados
df_alquileres['fecha_scrapping'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_scrapping, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 16: fecha_cambio_bna

In [522]:
# Verificamos los duplicados
df_alquileres['fecha_cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna fecha_cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 17: cambio_bna

In [523]:
# Verificamos los duplicados
df_alquileres['cambio_bna'].duplicated(keep="first").sum()

1027

*ACCION:*
<br>
*Entendemos que la columna cambio_bna, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

+ Columna 18: provincia

In [524]:
# Verificamos los duplicados
df_alquileres['provincia'].duplicated(keep="first").sum()

1024

*ACCION:*
<br>
*Entendemos que la columna provincia, puede tener valores iguales para las diferentes observaciones*
<br>
*Por lo cual, no eliminamos segun los repetidos basados en esta columna*

*OBSERVACION GENERAL*
<br>
*En general entendemos que la mayoria de las columnas podrian tener valores repetidos evaluandolas individualmente.*
<br>
*Pero si consideramos que la variable id_alojamiento identifica de manera biunivoca un alojamiento.*
<br>
*Podemos decir que no es lo mismo, un duplicado en la columna id_alojamiento(que no deberia tener valores repetidos), que un duplicado en la columna price_original,*
<br>
*(que podria perfectamente tener valores repetidos en la seria).*


**<h1>3. Análisis de datos faltantes y limpieza</h1>**

+ Mostrar el porcentaje de datos faltantes de cada columna

In [525]:
# Verificacion inicial del dataset
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

*En primera instancia observamos que solo nos quedan dos columnas con datos faltantes: rating y price_discounted*
<br>
*Para rating, tenenmos un 81.90 % de faltantes (842 de 1028).*
<br>
*Para rating, tenenmos un 18.58 % de faltantes (191 de 1028).*


+ ¿Cuáles son las 3 columnas con más datos faltantes?

*En nuestra observacion inicial, en principio tenemos dos columnas con datos faltantes solamente: rating y price_discounted*

**LIMPIEZA DE DE DATOS POR COLUMNAS**

In [526]:
df_alquileres.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   id_alojamiento    1028 non-null   int64  
 1   name              1028 non-null   object 
 2   category          1028 non-null   object 
 3   rating            842 non-null    object 
 4   city              1024 non-null   object 
 5   latitude          1028 non-null   float64
 6   longitude         1028 non-null   float64
 7   characteristics   1028 non-null   object 
 8   check_in          1028 non-null   object 
 9   check_out         1028 non-null   object 
 10  price_discounted  191 non-null    object 
 11  price_original    1028 non-null   object 
 12  price_qualifier   1028 non-null   object 
 13  cant_resultados   1028 non-null   int64  
 14  fecha_scrapping   1028 non-null   object 
 15  fecha_cambio_bna  1028 non-null   object 
 16  cambio_bna        1028 non-null   float64
 17  

+ Columna 0: id_alojamiento

In [527]:
df_alquileres['id_alojamiento'].nunique()

1028

In [528]:
df_alquileres['id_alojamiento'].isna().sum()

0

+ Columna 1: name

In [529]:
sorted(df_alquileres['name'].unique())

['"Alegria" vistas de ensueño en un lugar para enamorarse',
 '"Bathroom with LakeView Place and Retreat"',
 '"Casita" a estrenar, 21 m2 de luz',
 '"Cerros Nevados" departamento tipo loft.',
 '"Cipresales"     Cabaña  Villa la Angostura  Nqn',
 '"Doña Cota" - Amueblados Comahue',
 '"El Rancho" Sentirte en casa',
 '"La Encantada" en Villa los Coihues',
 '"La Posadita" Casa de Montaña, Miralejos',
 '"La casa de Cielo "',
 '"Los Amancays "\nCabaña de montaña',
 '"Tiny House de Ensueño"',
 '"Ty Bychan" \nLa Casita. Entre raíces galesas.',
 '0 - Pura Tranquilidad en el Corazón de Neuquén',
 '1- Pura Tranquilidad en el Corazón de Neuquén',
 '11-E Espectacular departamento Alto Neuquen',
 '2 - Pura tranquilidad en el corazón de Neuquén',
 '2 Amigos de viaje en un lugar comodo',
 '2 Habitaciones. 4 Personas en Casa con Pileta',
 '2 Personas. Lugar súper cómodo',
 '2 ambientes en el centro',
 '2x1 Central Family',
 '3 dormitorios. cama matrimonial + 3 de una plaza',
 '3- Pura tranquilidad en el 

+ Columna 2: category

In [530]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['category'].unique()

array(['entire_home', 'private_room', 'shared_room'], dtype=object)

In [531]:
# Verificamos que no tienen  na
df_alquileres['category'].isna().sum()

0

In [532]:
# Cambiamos el tipo de la serie
df_alquileres['category'] = df_alquileres['category'].astype('category')

+ Columna 3: rating

In [533]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['rating'].unique()

array([nan, '5,0 (6)', '4,91 (70)', 'Nuevo', '4,96 (95)', '4,86 (7)',
       '4,82 (17)', '4,96 (75)', '4,91 (67)', '4,57 (14)', '4,6 (5)',
       '4,94 (18)', '4,95 (22)', '4,92 (64)', '4,94 (109)', '4,77 (44)',
       '4,89 (35)', '5,0 (7)', '4,66 (53)', '5,0 (34)', '4,98 (93)',
       '4,96 (51)', '4,82 (66)', '4,8 (51)', '4,83 (36)', '4,9 (20)',
       '5,0 (17)', '4,65 (49)', '4,7 (33)', '4,89 (9)', '4,97 (90)',
       '5,0 (10)', '4,79 (30)', '4,86 (74)', '4,86 (44)', '4,63 (128)',
       '5,0 (3)', '4,88 (16)', '4,81 (32)', '5,0 (9)', '4,93 (27)',
       '4,92 (13)', '4,5 (4)', '4,77 (13)', '4,67 (12)', '4,9 (176)',
       '4,95 (19)', '4,8 (15)', '5,0 (14)', '4,79 (14)', '5,0 (4)',
       '5,0 (19)', '4,67 (6)', '4,82 (137)', '5,0 (11)', '4,77 (102)',
       '4,47 (55)', '4,76 (134)', '4,74 (19)', '4,96 (27)', '4,91 (11)',
       '4,93 (15)', '4,62 (13)', '4,55 (33)', '4,4 (5)', '4,72 (25)',
       '4,58 (76)', '4,73 (11)', '4,83 (60)', '4,93 (28)', '4,7 (10)',
       '4,76 (85

In [534]:
# Tratamiento de la columna rating
# Separamos los valores correspondientes para limpiarlos
df_rating= df_alquileres['rating'].str.split(pat="[ ]",expand=True).add_prefix('Columna')

# Renombramos las columnas
df_rating.rename(columns={"Columna0": "rating", "Columna1": "votaciones"}, inplace=True)

In [535]:
# Limpiamos los valores para la columna rating
df_rating['rating'] = df_rating['rating'].replace(',','.')

In [536]:
#  Rellenamos los valores para las columnas rating y votaciones
# Rellenamos con un valor promedio, pero establecemos que no tiene votaciones
df_rating.fillna({'rating':'3.00','votaciones':'0'},inplace= True)
# Quedo en la columna rating, con valor particular que debemos sustituir: Nuevo
df_rating['rating'] = np.where(df_rating['rating'].str.lower().str.contains('nuevo'), '3.00', df_rating['rating'])

In [537]:
# Reemplazamos la columna por punto para la columna rating y lo cambiamos de tipo
df_rating['rating'] = df_rating['rating'].str.replace(',','.')
df_rating['rating'] = df_rating['rating'].astype('float64')

In [538]:
# Limpiamos la columna de votaciones para cambiarle el tipo
df_rating['votaciones'] = df_rating['votaciones'].str.replace('(','').str.replace(')','')
df_rating['votaciones'] = df_rating['votaciones'].astype('int')

In [539]:
# Enriquecemos el dataframe de alquileres con las columnas de rating y votaciones
df_alquileres= df_alquileres.merge(df_rating,how='left',left_index=True,right_index=True,suffixes=('', '_y'))
df_alquileres[['rating_y','votaciones']].head()


,rating_y,votaciones
0,3.00,0
1,3.00,0
2,5.00,6
3,4.91,70
4,3.00,0


In [540]:
# Actualizamos las columnas del dataframe rating y votaciones
df_alquileres['rating']= df_alquileres['rating_y']
df_alquileres = df_alquileres.drop('rating_y',axis=1)
df_alquileres.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1028 entries, 0 to 1079
Data columns (total 19 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   id_alojamiento    1028 non-null   int64   
 1   name              1028 non-null   object  
 2   category          1028 non-null   category
 3   rating            1028 non-null   float64 
 4   city              1024 non-null   object  
 5   latitude          1028 non-null   float64 
 6   longitude         1028 non-null   float64 
 7   characteristics   1028 non-null   object  
 8   check_in          1028 non-null   object  
 9   check_out         1028 non-null   object  
 10  price_discounted  191 non-null    object  
 11  price_original    1028 non-null   object  
 12  price_qualifier   1028 non-null   object  
 13  cant_resultados   1028 non-null   int64   
 14  fecha_scrapping   1028 non-null   object  
 15  fecha_cambio_bna  1028 non-null   object  
 16  cambio_bna        1028 non-nu

+ Columna 4: city

In [541]:
# Vemos que tiene valores bien definidos, que no necesitan normalizar
df_alquileres['city'].unique()

array(['Puerto Madryn', 'Comodoro Rivadavia', 'Trelew', 'Esquel',
       'Puerto Piramides', 'Puerto Pirámides', 'Trevelin', 'Cholila',
       'Epuyén', 'Playa Unión', 'Lago Puelo', 'Playa Union', 'Rada Tilly',
       'Rawson', 'Biedma', 'Esquel,', 'Gaiman', 'El Hoyo', 'Cushamen',
       'Playa Magagna', 'Camarones', 'Biedma Department',
       'Gobernador Costa', 'Corcovado, Chubut', 'Neuquén', 'Neuquen',
       'BVH', 'Cipolletti', 'MDJ', 'Plottier', 'Confluencia',
       'Centenario', nan, 'Q8302ATE', 'JKB', 'Las Perlas', 'KCN',
       'Q8300MDJ', 'Confluencia Department', 'San Martín de Los Andes',
       'BGJ', 'Villa Traful', 'Villa La Angostura',
       'San Carlos de Bariloche', 'Bariloche', 'San Martin de los Andes',
       'Dina Huapi', 'Lácar', 'San Martín de los Andes', 'La Villa',
       'San Carlos de Bariloche-  Península de San Pedro',
       'Villa la Angostura ', 'Los Lagos', 'Puerto Manzano', 'Bariloche ',
       'San Carlos de Bariloche, Río Negro, AR', 'Viedma',
  

In [542]:
# Como vemos que varias ciudades son la misma pero con alguna diferencia tipografica, tratamos de normalizar los valores
df_alquileres['city'] = df_alquileres['city'].str.strip().str.upper()
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones
0,855050336912355203,North Beach,entire_home,3.00,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,3.00,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0
2,800749301724167838,Departamento Bella Vista,entire_home,5.00,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.91,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70
4,916738743930751566,Front Beach,entire_home,3.00,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0


In [543]:
# Quitamos los tildes
df_alquileres['city'] = df_alquileres['city'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_alquileres.head()

,id_alojamiento,name,category,rating,city,latitude,longitude,characteristics,check_in,check_out,price_discounted,price_original,price_qualifier,cant_resultados,fecha_scrapping,fecha_cambio_bna,cambio_bna,provincia,votaciones
0,855050336912355203,North Beach,entire_home,3.00,PUERTO MADRYN,-42.756301,-65.037517,"['2 camas', '1 dormitorio', '1 baño']",2023-07-06,2023-07-12,NaN,$38 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0
1,51484364,Hermoso Monoambiente con Vista al Mar y Pileta,entire_home,3.00,PUERTO MADRYN,-42.755775,-65.037265,"['2 camas', '1 dormitorio', '1 baño', '1 cama ...",2023-07-21,2023-07-26,$18 USD,$24 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0
2,800749301724167838,Departamento Bella Vista,entire_home,5.00,PUERTO MADRYN,-42.768007,-65.033108,"['3 camas', '1 dormitorio', '1 baño', 'Calific...",2023-08-01,2023-08-08,NaN,$43 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,6
3,50769917,New Lil’ & Cozy - 100mts de la playa Puerto Ma...,entire_home,4.91,PUERTO MADRYN,-42.777590,-65.025890,"['1 cama', '1 dormitorio', '1 baño', '1 cama q...",2023-07-09,2023-07-14,NaN,$41 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,70
4,916738743930751566,Front Beach,entire_home,3.00,PUERTO MADRYN,-42.770540,-65.030810,"['2 camas', 'Monoambiente', '1 baño']",2023-07-01,2023-07-06,NaN,$35 USD,noche,1000,2023-06-24,2023-05-01,239.919355,CHUBUT,0


In [544]:
# Recopilamos los datos de ciudades que tienen lugeres turisicos, segun la provincia, para tratar de agruparlas
aux_ciudades = pd.read_csv('Provincias_Ciudades.csv',sep= ';',encoding='latin1')
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.upper().str.strip()
aux_ciudades['ciudad'] = aux_ciudades['ciudad'].str.normalize(
    'NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
aux_ciudades.head()

,ciudad,provincia
0,ALTO RIO SENGUER,CHUBUT
1,BAHIA BUSTAMANTE,CHUBUT
2,CAMARONES,CHUBUT
3,CHOLILA,CHUBUT
4,COMODORO RIVADAVIA,CHUBUT


In [545]:
# Creamos una columna city2, que va a a guardar los valores resultantes para city
df_alquileres['city2'] = df_alquileres['city']


# Parametros para los cuales buscar la mejor aproximacion por fuzzy
provincias = ['TIERRA DEL FUEGO', 'NEUQUEN', 'RIO NEGRO', 'CHUBUT']
# Valres de match que aceptamos
aprobaciones = [80, 80, 80, 80]

# rr=0;
# iter=0
# Iteramos por indice, para provincias y aprobaciones
for indice in range((len(provincias))):
    # Iteramos  por cada ciudad segun la provincia
    for _, row in df_alquileres.loc[df_alquileres['provincia']==provincias[indice]].iterrows():
        ciudad_probar = str(row['city'])
        #aloj=-1
        #iter+=1
        max_coincidencia=0
        encontro = False
        # Recorremos la lista de ciudades de la provincia recabados para verificar
        for valor_correcto in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:            
            # Obtenemos el ratio de coincidencia
            coincidencia = fuzz.ratio(ciudad_probar, valor_correcto)
            #print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto} de {provincias[indice]}. Resultdo {coincidencia}")
            # Si el puntaje de la coincidencia es mayor que la vigente
            if coincidencia > max_coincidencia:
                # Guardar el valor correcto
                correccion = valor_correcto
                # Reemplazar el puntaje
                max_coincidencia = coincidencia                
                if (coincidencia > max_coincidencia) & (max_coincidencia >= aprobaciones[indice]):
                    # Guardar el valor correcto
                    correccion = valor_correcto
                    # Reemplazar el puntaje
                    max_coincidencia = coincidencia
                    # Reemplazo por el mejor valor aproximado, en la columna auxiliar
                    df_alquileres['city2']=correccion
                    # Aviso que encontre algo
                    encontro = True
        # Si no encontre una coincidencia suficiente, cambio de macheo
        if( not encontro):
            # Recorremos la lista de departamentos
            max_coincidencia_parcial = 0            
            for valor_correcto_parcial in aux_ciudades.loc[aux_ciudades['provincia'] == provincias[indice]]['ciudad']:
                # Obtenemos el ratio de coincidencia
                coincidencia_parcial = fuzz.partial_ratio(ciudad_probar, valor_correcto_parcial)
                # print(f"Iteraccion: {iter},id_Alojamiento:{row['id_alojamiento']} index: {_} ==== Comparo {ciudad_probar} de {str(row['provincia'])} , con {valor_correcto_parcial} de {provincias[indice]}. Resultdo {coincidencia_parcial}")
                # Si el puntaje de la coincidencia es mayor que la vigente
                if coincidencia_parcial > max_coincidencia_parcial:
                    # Guardar el valor correcto
                    correccion_parcial = valor_correcto_parcial
                    # Reemplazar el puntaje
                    max_coincidencia_parcial = coincidencia_parcial
                    if (coincidencia_parcial > max_coincidencia_parcial) & (max_coincidencia_parcial >= aprobaciones[indice]):
                        # Guardar el valor correcto
                        correccion_parcial = valor_correcto_parcial
                        # Reemplazar el puntaje
                        max_coincidencia_parial = coincidencia_parcial                        
                        # Asigno el mejor valor posible encontrado
                        df_alquileres['city2']=correccion_parcial
                        encontro = True
                        
                        
# En caso de que no se cumpla con las condiciones del macheo, mantiene el valor para ciudad inicial
            

In [546]:
# Tenemos cuatro alojamientos que no tienen ciudad ingresada
df_alquileres['city'].isna().sum()

4

In [547]:
# Consideramos que la ciudad es un dato importante, y por la cantidad la accion a tomar es elimnar esos 4 registros
df_alquileres.dropna(subset=['city'],inplace= True)

In [548]:
df_alquileres['city']=df_alquileres['city2']
df_alquileres = df_alquileres.drop('city2', axis=1)

In [549]:
df_alquileres['city'].unique()

array(['PUERTO MADRYN', 'COMODORO RIVADAVIA', 'TRELEW', 'ESQUEL',
       'PUERTO PIRAMIDES', 'TREVELIN', 'CHOLILA', 'EPUYEN', 'PLAYA UNION',
       'LAGO PUELO', 'RADA TILLY', 'RAWSON', 'BIEDMA', 'ESQUEL,',
       'GAIMAN', 'EL HOYO', 'CUSHAMEN', 'PLAYA MAGAGNA', 'CAMARONES',
       'BIEDMA DEPARTMENT', 'GOBERNADOR COSTA', 'CORCOVADO, CHUBUT',
       'NEUQUEN', 'BVH', 'CIPOLLETTI', 'MDJ', 'PLOTTIER', 'CONFLUENCIA',
       'CENTENARIO', 'Q8302ATE', 'JKB', 'LAS PERLAS', 'KCN', 'Q8300MDJ',
       'CONFLUENCIA DEPARTMENT', 'SAN MARTIN DE LOS ANDES', 'BGJ',
       'VILLA TRAFUL', 'VILLA LA ANGOSTURA', 'SAN CARLOS DE BARILOCHE',
       'BARILOCHE', 'DINA HUAPI', 'LACAR', 'LA VILLA',
       'SAN CARLOS DE BARILOCHE-  PENINSULA DE SAN PEDRO', 'LOS LAGOS',
       'PUERTO MANZANO', 'SAN CARLOS DE BARILOCHE, RIO NEGRO, AR',
       'VIEDMA', 'LACAR DEPARTMENT',
       'VILLA LA ANGOSTURA, NEUQUEN ARGENTINA', 'LAS GRUTAS', 'EL BOLSON',
       'JUNIN DE LOS ANDES', 'LOS LAGOS DEPARTMENT', 'VILLA LLA

+ Columna 5: latitude

In [550]:
df_alquileres['latitude'].info()

<class 'pandas.core.series.Series'>
Index: 1024 entries, 0 to 1079
Series name: latitude
Non-Null Count  Dtype  
--------------  -----  
1024 non-null   float64
dtypes: float64(1)
memory usage: 16.0 KB


In [551]:
df_alquileres['latitude'].unique()

array([-42.7563015 , -42.75577473, -42.76800685, -42.77759   ,
       -42.77054   , -42.76833   , -42.78191   , -42.76473   ,
       -42.7716337 , -42.7538    , -42.76864   , -45.81441   ,
       -42.76506   , -42.76887   , -43.25534   , -42.91027   ,
       -42.76985   , -42.767826  , -42.77496   , -45.86703   ,
       -42.56882   , -43.27444   , -43.25614   , -42.77106   ,
       -45.86496   , -42.91138   , -42.77884   , -42.9148784 ,
       -42.9132481 , -42.91526   , -42.9074621 , -42.7687445 ,
       -42.56749   , -43.09089   , -43.25905   , -42.76854884,
       -45.86038   , -42.91053   , -42.772861  , -42.65054   ,
       -42.76724   , -43.2591    , -42.77329   , -42.91809   ,
       -42.92146   , -42.76916   , -42.752541  , -42.5117    ,
       -42.77835745, -43.25566171, -42.7756997 , -42.90797   ,
       -42.22750854, -42.77086   , -42.91956   , -42.99143   ,
       -42.56918   , -42.77094   , -42.57219   , -42.56978   ,
       -43.32639   , -43.25009918, -42.77208   , -42.76

In [552]:
# No tenemos valores nulos
df_alquileres['latitude'].isna().unique()

array([False])

+ Columna 6: longitude

In [553]:
df_alquileres['longitude'].info()

<class 'pandas.core.series.Series'>
Index: 1024 entries, 0 to 1079
Series name: longitude
Non-Null Count  Dtype  
--------------  -----  
1024 non-null   float64
dtypes: float64(1)
memory usage: 16.0 KB


In [554]:
# No tenemos valores nulos
# df_alquileres['longitude'].isna().unique()
df_alquileres['longitude'].unique()

array([-65.0375173 , -65.03726515, -65.03310833, ..., -68.30871   ,
       -68.3178    , -68.27647   ])